# Border Generation

For TEM (Transmission Electron Microscope) and SEM (Scanning Electron Microscope) Images.

In [27]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

from pathlib import Path

In [28]:
# Constants
IMAGES_DIR_PATH = "/home/pooh555/coding/image_analysis/TEM_NW/20250709-TEM-Yutichai/E8-NW/" # Raw images directory
OUTPUT_DIR_PATH = "./outputs/border_generation/"    # Saved images directory
INPUT_FILE_EXTENSION = "tif"   # Use '*' to include all file extensions
OUTPUT_FILE_EXTENSION = "tif"  # File extension for processed images

### Exploratory Data Analysis

Conduct EDA to examine the properties of the samples.

In [ ]:
# Store all images in the direcotry in "images" list
images = []
file_names = []

for file in glob.glob(IMAGES_DIR_PATH + "*." + INPUT_FILE_EXTENSION):
        file_name = Path(file).stem # Extract the original file name (extension excluded)
        images.append(cv2.imread(IMAGES_DIR_PATH + file_name + '.' + INPUT_FILE_EXTENSION, cv2.IMREAD_COLOR_BGR))   # Read and store all images in BGR format
        file_names.append(file_name)

In [ ]:
# Check for imparity
print(len(images))
print(len(file_names))

0
0


In [ ]:
# Displaying sample images
def display_images(images):
    print("Displaying images in: " + IMAGES_DIR_PATH)

    num_images_in_figure = 9    # Number of images in the figure
    count = 1   # Image counter

    plt.figure(figsize=(10, 10))    # Figure size

    # Plotting all images
    for image in images:
        if (count > num_images_in_figure):
            break

        plt.subplot(3, 3, count)
        plt.imshow(image)  
        plt.axis('off') 
        plt.title(file_name, fontsize=9)

        count += 1

    plt.tight_layout()
    plt.show()

In [ ]:
display_images(images)

Displaying images in: /home/pooh555/coding/TEM_image_analysis/TEM_NW/20250709-TEM-Yutichai/E8-NW/


<Figure size 1000x1000 with 0 Axes>

### Data Preparation

Adjusting constrast, brightness, etc to facilitate border marking process.

In [ ]:
# Image Properties Modification Function

# Note: The order of processing layers in this function matters

def modify_image(image, alpha=1.5, beta=10, guassian_kernel_size=5):
    processed_image = cv2.GaussianBlur(image, (guassian_kernel_size, guassian_kernel_size), 0)
    processed_image = cv2.convertScaleAbs(processed_image, alpha=alpha, beta=beta)

    return processed_image

In [ ]:
# Modify image properties
modified_images = []

for image in images:
    modified_images.append(modify_image(image))


In [ ]:
display_images(modified_images)

Displaying images in: /home/pooh555/coding/TEM_image_analysis/TEM_NW/20250709-TEM-Yutichai/E8-NW/


<Figure size 1000x1000 with 0 Axes>

### Border Synthesis

In [ ]:
# Border Generation Function
def generate_border(image, border_size=5, n_erosions=1):
    # Ensure the input image is color (BGR)
    if len(image.shape) == 2:   # Convert greyscale to BGR
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    elif image.shape[2] == 4:   # Convert RGBA to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)

    greyscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   # Convert the image to grayscale for erosion/dilation operations
    
    # Image erosion
    erosion_kernel = np.ones((3, 3), np.uint8)
    eroded_image_greyscale = cv2.erode(greyscale_image, erosion_kernel, iterations=n_erosions)

    # Image dilation
    kernel_size = 2 * border_size + 1   
    dilation_kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated_image_greyscale = cv2.dilate(eroded_image_greyscale, dilation_kernel, iterations=1)

    # Create a mask for the border    
    border_color = (0, 0, 255) # Define the border color (BGR)
    border_mask = ((dilated_image_greyscale == 255) & (eroded_image_greyscale < 255))    # The border region are pixels where dilated_image_gray is 255 but eroded_image_greyscale is not 255 (or 0)
    image_with_border = image.copy()    # Create a copy of the original color image to draw the border on 
    image_with_border[border_mask] = border_color  # Apply the border mask

    return image_with_border

In [ ]:
# Iterating over all images in the desired directory
print("Processing images in: " + IMAGES_DIR_PATH)

for image, file_name in zip(modified_images, file_names):
    processed_image = generate_border(image)    # Generated the objects borders

    cv2.imwrite(OUTPUT_DIR_PATH + file_name + '.' + OUTPUT_FILE_EXTENSION, processed_image)   # Save the processed image
    print("Finished processing: "+ file_name + '.' + INPUT_FILE_EXTENSION)

Processing images in: /home/pooh555/coding/TEM_image_analysis/TEM_NW/20250709-TEM-Yutichai/E8-NW/


# Test section

The code snippets below are solely used for testing purposes, and it can be ignored.

In [ ]:
# Border Generation Function (Debugging version)
def generate_border_debug(image, border_size=5, n_erosions=1):
    # Ensure the input image is color (BGR)
    if len(image.shape) == 2:   # Convert greyscale to BGR
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    elif image.shape[2] == 4:   # Convert RGBA to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)

    greyscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   # Convert the image to grayscale for erosion/dilation operations
    
    # Image erosion
    erosion_kernel = np.ones((3, 3), np.uint8)
    eroded_image_greyscale = cv2.erode(greyscale_image, erosion_kernel, iterations=n_erosions)

    # Image dilation
    kernel_size = 2 * border_size + 1   
    dilation_kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated_image_grayscale = cv2.dilate(eroded_image_greyscale, dilation_kernel, iterations=1)
    
    plt.imshow(dilated_image_grayscale, cmap="gray")
    plt.show()

    # Create a mask for the border   
    border_color = (0, 0, 255) # Define the border color (BGR) 
    border_mask = ((dilated_image_grayscale == 255) & (eroded_image_greyscale < 255))    # The border region are pixels where dilated_image_gray is 255 but eroded_image_greyscale is not 255 (or 0)
    image_with_border = image.copy()    # Create a copy of the original color image to draw the border on 
    image_with_border[border_mask] = border_color  # Apply the border mask
    image_with_border_rgb = cv2.cvtColor(image_with_border, cv2.COLOR_BGR2RGB)  # Convert BRG to RGB

    plt.imshow(image_with_border_rgb)
    plt.show()

    return image_with_border

In [ ]:
image = cv2.imread(IMAGES_DIR_PATH + "OneView 200kV 25kX E8-NW Yutichai 0003.tif", cv2.IMREAD_GRAYSCALE)
modified_image = modify_image(image, alpha=2.0, beta=2.5, guassian_kernel_size=11)
processed_image = generate_border_debug(modified_image)

cv2.imwrite(OUTPUT_DIR_PATH + "sample_output" + '.' + OUTPUT_FILE_EXTENSION, processed_image)

[ WARN:0@16.694] global loadsave.cpp:268 findDecoder imread_('/home/pooh555/coding/TEM_image_analysis/TEM_NW/20250709-TEM-Yutichai/E8-NW/OneView 200kV 25kX E8-NW Yutichai 0003.tif'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /home/conda/feedstock_root/build_artifacts/libopencv_1743559199045/work/modules/imgproc/src/smooth.dispatch.cpp:618: error: (-215:Assertion failed) !_src.empty() in function 'GaussianBlur'
